# Understand the Basic of Tool Calling

Import libraries, env, and llm model

In [45]:
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, ToolMessage
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
rapid_api_key = os.getenv("RAPID_API_KEY")
llm = ChatGroq(temperature=0, model_name="llama3-70b-8192", groq_api_key = groq_api_key)
llm2 = ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0)

Define tools

In [57]:
@tool
def say_hi(name:str) -> str:
   "Use this tool to greet people"
   return f"Hi {name}"

@tool
def say_bye(name:str) -> str:
   "Use this tool to bid farewell to people"
   return f"Bye {name}"

tools = [say_hi, say_bye]
llm_with_tools = llm.bind_tools(tools)

In [58]:
query = "My sister named Gisel is coming to my house, what should I say?"

In [59]:
messages = [(HumanMessage(query))]
messages

[HumanMessage(content='My sister named Gisel is coming to my house, what should I say?')]

In [60]:
output = llm_with_tools.invoke(messages)
messages.append(output)
messages

[HumanMessage(content='My sister named Gisel is coming to my house, what should I say?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h3e5', 'function': {'arguments': '{"name":"Gisel"}', 'name': 'say_hi'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.125714286, 'completion_tokens': 44, 'prompt_time': 0.199227426, 'prompt_tokens': 914, 'queue_time': None, 'total_time': 0.324941712, 'total_tokens': 958}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-44e208e6-6ff5-4c9f-aee5-579edc1eb21a-0', tool_calls=[{'name': 'say_hi', 'args': {'name': 'Gisel'}, 'id': 'call_h3e5'}])]

Based on the latest langchain update, we can access tool_calls ( to access which tool is going to be used, and what is the argument)

In [61]:
tool_calls = output.tool_calls
print(tool_calls)
print('selected tool:', tool_calls[0]['name'])
print('argument that will be passed to the tool:', tool_calls[0]['args'])

[{'name': 'say_hi', 'args': {'name': 'Gisel'}, 'id': 'call_h3e5'}]
selected tool: say_hi
argument that will be passed to the tool: {'name': 'Gisel'}


In [62]:
tool_mapping = {'say_hi':say_hi, 'say_bye': say_bye} # mapping between tool name and defined tool function
selected_tool = tool_mapping[tool_calls[0]['name']] # used to get the selected tool
tool_output = selected_tool.invoke(tool_calls[0]['args']) # invoke the selected tool with the argument
print(tool_output)

Hi Gisel


In [63]:
messages.append(ToolMessage(tool_output, tool_call_id = tool_calls[0]['id']))
messages

[HumanMessage(content='My sister named Gisel is coming to my house, what should I say?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_h3e5', 'function': {'arguments': '{"name":"Gisel"}', 'name': 'say_hi'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_time': 0.125714286, 'completion_tokens': 44, 'prompt_time': 0.199227426, 'prompt_tokens': 914, 'queue_time': None, 'total_time': 0.324941712, 'total_tokens': 958}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_87cbfbbc4d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-44e208e6-6ff5-4c9f-aee5-579edc1eb21a-0', tool_calls=[{'name': 'say_hi', 'args': {'name': 'Gisel'}, 'id': 'call_h3e5'}]),
 ToolMessage(content='Hi Gisel', tool_call_id='call_h3e5')]

In [64]:
output = llm_with_tools.invoke(messages)
output

AIMessage(content='You can simply respond with the result: Hi Gisel', response_metadata={'token_usage': {'completion_time': 0.034285714, 'completion_tokens': 12, 'prompt_time': 0.158173437, 'prompt_tokens': 982, 'queue_time': None, 'total_time': 0.192459151, 'total_tokens': 994}, 'model_name': 'llama3-70b-8192', 'system_fingerprint': 'fp_753a4aecf6', 'finish_reason': 'stop', 'logprobs': None}, id='run-2924aed6-5318-42b7-86f0-ccd0c415fd4f-0')

In [69]:
output.content

'You can simply respond with the result: Hi Gisel'

# create_tool_calling_agent

In [68]:
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

@tool
def say_hi(name:str) -> str:
   "Use this tool to greet people"
   return f"Hi {name}"

@tool
def say_bye(name:str) -> str:
   "Use this tool to bid farewell to people"
   return f"Bye {name}"

tools = [say_hi, say_bye]

# Define the prompt 
prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

query = "My sister named Gisel is coming to my house, what should I say?"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: My sister named Gisel is coming to my house, what should I say?
Answer: You can say "Hi Gisel" to your sister when she arrives at your house.


If you want to see what happened inside, we could set verbose = True

In [70]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)
agent_executor.invoke({"input": query})



> Entering new AgentExecutor chain...

Invoking: `say_hi` with `{'name': 'Gisel'}`


Hi GiselYou can respond directly to your sister: "Hi Gisel"

> Finished chain.


{'input': 'My sister named Gisel is coming to my house, what should I say?',
 'output': 'You can respond directly to your sister: "Hi Gisel"'}

Working with real use case

In [33]:
import requests
import json
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

@tool()
def get_company_profile(stock:str) -> str:
    """Get detail profile such as company name, sector name, primary name, number of employees of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-profile"

    querystring = {"symbols":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")
    
    return result

@tool()
def get_competitors(stock:str) -> str:
    """Get peers or competitors of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-peers"

    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")
    
    return result

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile, get_competitors]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

Note: Different models yield different outputs, so it would be wise to experiment with open-source models if you're exploring. However, for production purposes, I highly suggest using GPT (as far as I've experimented, GPT is still the best in tool calling) or you could also use a fine-tuned model. Let's see the difference below!

- Using llama3

In [24]:
query = "I'm an investor, what is AAPL"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: I'm an investor, what is AAPL


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated

In [26]:
result['output']

'<|start_header_id|>assistant<|end_header_id|>\n\nassistant<|end_header_id|>\n\nassistant<|end_header_id|>\n\nassistant<|end_header_id|><|start_header_id|>assistant<|end_header_id|>\n\nassistant<|end_header_id|><|start_header_id|>assistant<|end_header_id|>\n\nassistant'

- Using gpt-3.5-turbo

In [29]:
query = "I'm an investor, what is AAPL"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: I'm an investor, what is AAPL


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated

In [31]:
print(result['output'])

AAPL is the stock ticker symbol for Apple Inc. Apple designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers a range of products including iPhone, Mac, iPad, AirPods, Apple TV, Apple Watch, Beats products, and HomePod. Apple also provides services such as AppleCare support, cloud services, App Store, Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, Apple Card, and Apple Pay.

Here are some key details about Apple Inc.:
- Sector: Information Technology
- Industry: Technology Hardware, Storage and Peripherals
- Number of Employees: 161,000
- Year Founded: 1976
- Headquarters: Cupertino, California
- Website: [www.apple.com](https://www.apple.com)
- Market Cap: $3.21 trillion
- Last Close Price: $213.25
- P/E Ratio (Forward): 32.35

If you would like more information or have any specific questions about Apple Inc., feel free to ask!


In [34]:
query = "I'm an investor, give me information about AAPL, and also give me the peers"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: I'm an investor, give me information about AAPL, and also give me the peers


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness serv

In [35]:
print(result['output'])

### Company Profile:
- **Company Name:** Apple Inc.
- **Sector:** Information Technology
- **Primary Industry:** Technology Hardware, Storage and Peripherals
- **Number of Employees:** 161,000
- **Year Founded:** 1976
- **Market Cap:** $3,205,896,523,740
- **EPS:** $6.45
- **Dividend Yield:** 0.47%
- **Website:** [www.apple.com](https://www.apple.com)

### Peers/Competitors:
1. **Dell Technologies Inc. (DELL)**
   - ![Dell Technologies Inc. Logo](https://static.seekingalpha.com/cdn/iex/stable/stock/dell/logo?asImage=true)
   - **Exchange:** NYSE

2. **Xiaomi Corporation (XIACY)**
   - ![Xiaomi Corporation Logo](https://static.seekingalpha.com/cdn/iex/stable/stock/xiacy/logo?asImage=true)
   - **Exchange:** Pink Current Info

3. **Super Micro Computer, Inc. (SMCI)**
   - ![Super Micro Computer, Inc. Logo](https://static.seekingalpha.com/cdn/iex/stable/stock/smci/logo?asImage=true)
   - **Exchange:** NASDAQ

4. **HP Inc. (HPQ)**
   - ![HP Inc. Logo](https://static.seekingalpha.com/cdn/ie

# Tool Decorator + Pydantic

In most cases, we'll need Pydantic to validate our input. For instance, for tickers, the format typically consists of a maximum of 4 characters, such as AAPL, META, GOGL, etc. Therefore, we can logically validate whether the user input is in the correct format. If not, we can raise an error and avoid processing it further. This approach enhances the efficiency of our system in terms of cost and execution time.

In [36]:
query = "I'm an investor, what is AAPLLLLLLLLL"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: I'm an investor, what is AAPLLLLLLLLL


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPLLLLLLLLL'}`


{"data": [{"id": "AAPLLLLLLLLL", "tickerId": null, "attributes": {"longDesc": null, "sectorname": null, "sectorgics": null, "primaryname": null, "primarygics": null, "numberOfEmployees": null, "yearfounded": null, "streetaddress": null, "streetaddress2": null, "streetaddress3": null, "streetaddress4": null, "city": null, "state": null, "zipcode": null, "country": null, "officephonevalue": null, "webpage": null, "companyName": null, "marketCap": null, "totalEnterprise": null, "totAnalystsRecommendations": null, "fy1UpRevisions": null, "fy1DownRevisions": null, "divYield": null, "eps": null, "lastDaily": null, "estimateEps": null, "debtEq": null, "totDebtCap": null, "ltDebtEquity": null, "ltDebtCap": null, "totLiabTotAssets": null, "impliedMarketCap": null, "shortIntPctFloat": null, "divTimeFrame": null, "divRate": null, "peRatioFwd":

In [40]:
import requests
import json
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from pydantic.v1 import BaseModel, Field, validator
from datetime import datetime

# add pydantic
class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for, should only contain up to 4 characters")

# add args_schema=SearchInput inside of the tool decorator 
@tool(args_schema=SearchInput)
def get_company_profile(stock:str) -> str:
    """Get detail profile such as company name, sector name, primary name, number of employees of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-profile"

    querystring = {"symbols":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")
    
    return result

@tool(args_schema=SearchInput)
def get_competitors(stock:str) -> str:
    """Get peers or competitors of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-peers"

    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")
    
    return result

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile, get_competitors]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [41]:
query = "I'm an investor, what is AAPLLLLLLL"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: I'm an investor, what is AAPLLLLLLL


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a c

gpt-3.5-turbo is good enough in understanding the requirement, so it will auto generate that AAPLLLLLL means AAPL. but how if we want more control?

In [42]:
import requests
import json
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from pydantic.v1 import BaseModel, Field, validator
from datetime import datetime

# add pydantic
class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for")

    @validator('stock')
    def validate_stock(cls, v):
        if not v.isalpha() or len(v) > 4:
            raise ValueError('Stock ticker should only contain up to 4 alphabetic characters')
        return v

# add args_schema=SearchInput inside of the tool decorator 
@tool(args_schema=SearchInput)
def get_company_profile(stock:str) -> str:
    """Get detail profile such as company name, sector name, primary name, number of employees of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-profile"

    querystring = {"symbols":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")
    
    return result

@tool(args_schema=SearchInput)
def get_competitors(stock:str) -> str:
    """Get peers or competitors of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-peers"

    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ValueError(f"No data for {stock}")
    
    return result

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile, get_competitors]
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

In [43]:
query = "I'm an investor, what is AAPLLLLLLL"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: I'm an investor, what is AAPLLLLLLL


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPLLLLLLL'}`




ValidationError: 1 validation error for SearchInput
stock
  Stock ticker should only contain up to 4 alphabetic characters (type=value_error)

# Structured Tools

https://blog.langchain.dev/structured-tools/

If we raise a ValueError, our system will break, halting the process with an error in the middle. For example, if it's a chatbot, it might suddenly stop without providing any explanation. Therefore, we require **error handling**. Langchain already provides us with ToolException, but it can only be accessed through Structured Tools or the base class. We cannot implement ToolException using the tool decorator, illustrating why @tool is not always sufficient. For greater flexibility, we should use StructuredTools

1. change ValueError to ToolException
2. handle_tool_error = True

In [51]:
from pydantic.v1 import BaseModel, Field, validator
from datetime import datetime
from langchain_core.tools import StructuredTool
from langchain_core.tools import ToolException

# change ValueError to ToolException
class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for")
    
    @validator('stock')
    def validate_stock(cls, v):
        if not v.isalpha() or len(v) > 4:
            raise ToolException('Stock ticker should only contain up to 4 alphabetic characters')
        return v

def get_company_profile(stock:str) -> str:
    """Get detail profile such as company name, sector name, primary name, number of employees of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-profile"

    querystring = {"symbols":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data for {stock}")
    
    return result

def get_competitors(stock:str) -> str:
    """Get peers or competitors of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-peers"

    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data for {stock}")
    
    return result


get_company_profile_tool = StructuredTool.from_function(
    func=get_company_profile,
    args_schema= SearchInput,
    handle_tool_error=True, # add this
)

get_competitors_tool = StructuredTool.from_function(
    func=get_competitors,
    args_schema= SearchInput,
    handle_tool_error=True, # add this
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile_tool, get_competitors_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)


In [52]:
query = "I'm an investor, what is AAPLLLLLLL"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: I'm an investor, what is AAPLLLLLLL


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPLLLLLLL'}`


Stock ticker should only contain up to 4 alphabetic charactersI'm sorry, but the stock ticker should only contain up to 4 alphabetic characters. Could you please provide me with the correct stock ticker you are interested in?

> Finished chain.
Answer: I'm sorry, but the stock ticker should only contain up to 4 alphabetic characters. Could you please provide me with the correct stock ticker you are interested in?


# Adding Memory

As humans, memory is crucial. Similarly, in agents, we can also incorporate memory to provide context for ongoing conversations.

## Use case 1 without memory

In [53]:
query = "Hey, I'm Wilsen"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: Hey, I'm Wilsen


> Entering new AgentExecutor chain...
Hello Wilsen! How can I assist you today?

> Finished chain.
Answer: Hello Wilsen! How can I assist you today?


In [55]:
query = "What's my name?"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: What's my name?


> Entering new AgentExecutor chain...
I'm sorry, but I don't have access to personal information like your name. How can I assist you today?

> Finished chain.
Answer: I'm sorry, but I don't have access to personal information like your name. How can I assist you today?


## Use case 2 without memory

In [56]:
query = "Tell me about AAPL"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: Tell me about AAPL


> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening 

In [57]:
query = "How about the competitor?"
print("Question:", query)
result = agent_executor.invoke({"input": query})
print("Answer:",result['output'])

Question: How about the competitor?


> Entering new AgentExecutor chain...
Sure, I can help you with that. Could you please provide me with the stock ticker of the company for which you would like to know the competitors?

> Finished chain.
Answer: Sure, I can help you with that. Could you please provide me with the stock ticker of the company for which you would like to know the competitors?


## How to add?

In [ ]:
from pydantic.v1 import BaseModel, Field, validator
from datetime import datetime
from langchain_core.tools import StructuredTool
from langchain_core.tools import ToolException

class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for")
    
    @validator('stock')
    def validate_stock(cls, v):
        if not v.isalpha() or len(v) > 4:
            raise ToolException('Stock ticker should only contain up to 4 alphabetic characters')
        return v

def get_company_profile(stock:str) -> str:
    """Get detail profile such as company name, sector name, primary name, number of employees of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-profile"

    querystring = {"symbols":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data for {stock}")
    
    return result

def get_competitors(stock:str) -> str:
    """Get peers or competitors of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-peers"

    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data for {stock}")
    
    return result


get_company_profile_tool = StructuredTool.from_function(
    func=get_company_profile,
    args_schema= SearchInput,
    handle_tool_error=True, # add this
)

get_competitors_tool = StructuredTool.from_function(
    func=get_competitors,
    args_schema= SearchInput,
    handle_tool_error=True, # add this
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile_tool, get_competitors_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)


In [58]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("placeholder", "{history}"), # add this or could be: MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile_tool, get_competitors_tool]

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True)

store = {} # This dictionary acts like a dummy database that save the message history
print("History:", store)

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_executor_w_memory = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)


History: {}


## Use case 1 with memory

In [59]:
agent_executor_w_memory.invoke(
    {"input": "Hey, I'm Wilsen"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run eed92e66-03c5-4927-bf42-818c9d34a5f4 not found for run a1121e8d-ed4d-4ad6-b47c-5e6c286eb694. Treating as a root run.




> Entering new AgentExecutor chain...
Hello Wilsen! How can I assist you today?

> Finished chain.


{'input': "Hey, I'm Wilsen",
 'history': [],
 'output': 'Hello Wilsen! How can I assist you today?'}

In [60]:
agent_executor_w_memory.invoke(
    {"input": "What's my name?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run c164461b-4ab8-42ce-8e4a-4883511265dc not found for run 9a3b0bfe-2f21-43fb-8177-18cae7e8334f. Treating as a root run.




> Entering new AgentExecutor chain...
Your name is Wilsen. How can I help you today, Wilsen?

> Finished chain.


{'input': "What's my name?",
 'history': [HumanMessage(content="Hey, I'm Wilsen"),
  AIMessage(content='Hello Wilsen! How can I assist you today?')],
 'output': 'Your name is Wilsen. How can I help you today, Wilsen?'}

## Use case 2 with memory

In [61]:
agent_executor_w_memory.invoke(
    {"input": "Tell me about AAPL"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 8da7f55e-664b-4f87-97f8-578e333151e4 not found for run 1e76972a-e3e2-4ea8-a229-39ffb23379ce. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand rad

{'input': 'Tell me about AAPL',
 'history': [HumanMessage(content="Hey, I'm Wilsen"),
  AIMessage(content='Hello Wilsen! How can I assist you today?'),
  HumanMessage(content="What's my name?"),
  AIMessage(content='Your name is Wilsen. How can I help you today, Wilsen?')],
 'output': "Apple Inc. (AAPL) is a technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. Here are some key details about Apple Inc.:\n\n- **Sector:** Information Technology\n- **Primary Industry:** Technology Hardware, Storage and Peripherals\n- **Number of Employees:** 161,000\n- **Year Founded:** 1976\n- **Headquarters:** Cupertino, California\n- **Website:** [www.apple.com](https://www.apple.com)\n- **Market Cap:** $3.21 trillion\n- **Current Stock Price:** $213.25\n- **PE Ratio (Forward):** 32.35\n- **Dividend Yield:** 0.47%\n- **Analysts Recommendations:** 44\n- **EPS (Earnings Per Share):** $6.45\n\nApple offers a range of pro

In [62]:
agent_executor_w_memory.invoke(
    {"input": "How about the competitor?"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 8104f7c2-a850-4963-b90f-cca470a282ad not found for run bde445bd-49dc-4bad-9e63-2760a9f46fb0. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `get_competitors` with `{'stock': 'AAPL'}`


{"data": [{"id": "569700", "type": "ticker", "attributes": {"slug": "dell", "iexSlug": "dell", "name": "DELL", "companyName": "Dell Technologies Inc.", "equityType": "stocks", "indexGroup": null, "currency": "USD", "tradingViewSlug": "NYSE:DELL", "exchange": "NYSE", "exchangeDescription": null, "company": "Dell Technologies Inc.", "isBdc": false, "visible": true, "searchable": true, "private": false, "pending": false, "isDefunct": false, "followersCount": 92184, "fundTypeId": 0, "articleRtaCount": 8, "newsRtaCount": 28, "divYieldType": "forward", "primaryEpsConsensusMeanType": "normalized", "mergedOn": null, "isReit": false}, "relationships": {"sector": {"data": {"id": "45", "type": "sector"}}, "subIndustry": {"data": {"id": "45202030", "type": "subIndustry"}}}, "meta": {"companyLogoUrl": "https://static.seekingalpha.com/cdn/iex/stable/stock/dell/logo?asImage=true", "companyLogoUrlLight": "h

{'input': 'How about the competitor?',
 'history': [HumanMessage(content="Hey, I'm Wilsen"),
  AIMessage(content='Hello Wilsen! How can I assist you today?'),
  HumanMessage(content="What's my name?"),
  AIMessage(content='Your name is Wilsen. How can I help you today, Wilsen?'),
  HumanMessage(content='Tell me about AAPL'),
  AIMessage(content="Apple Inc. (AAPL) is a technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. Here are some key details about Apple Inc.:\n\n- **Sector:** Information Technology\n- **Primary Industry:** Technology Hardware, Storage and Peripherals\n- **Number of Employees:** 161,000\n- **Year Founded:** 1976\n- **Headquarters:** Cupertino, California\n- **Website:** [www.apple.com](https://www.apple.com)\n- **Market Cap:** $3.21 trillion\n- **Current Stock Price:** $213.25\n- **PE Ratio (Forward):** 32.35\n- **Dividend Yield:** 0.47%\n- **Analysts Recommendations:** 44\n- **EPS

# Adding Intermediate Steps in Memory

In [85]:
from pydantic.v1 import BaseModel, Field, validator
from datetime import datetime
from langchain_core.tools import StructuredTool
from langchain_core.tools import ToolException
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

class SearchInput(BaseModel):
    stock: str = Field(description="Stock ticker to search for")
    
    @validator('stock')
    def validate_stock(cls, v):
        if not v.isalpha() or len(v) > 4:
            raise ToolException('Stock ticker should only contain up to 4 alphabetic characters')
        return v

def get_company_profile(stock:str) -> str:
    """Get detail profile such as company name, sector name, primary name, number of employees of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-profile"

    querystring = {"symbols":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data for {stock}")
    
    return result

def get_competitors(stock:str) -> str:
    """Get peers or competitors of a stock"""

    api_key = rapid_api_key
    url = "https://seeking-alpha.p.rapidapi.com/symbols/get-peers"

    querystring = {"symbol":stock.lower()}

    headers = {
        "x-rapidapi-key": api_key,
        "x-rapidapi-host": "seeking-alpha.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    
    if response.status_code == 200:
        data = response.json()
        result = json.dumps(data)
    else:
        raise ToolException(f"No data for {stock}")
    
    return result


get_company_profile_tool = StructuredTool.from_function(
    func=get_company_profile,
    args_schema= SearchInput,
    handle_tool_error=True, # add this
)

get_competitors_tool = StructuredTool.from_function(
    func=get_competitors,
    args_schema= SearchInput,
    handle_tool_error=True, # add this
)


prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    ("placeholder", "{history}"), # add this or could be: MessagesPlaceholder(variable_name="history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

tools = [get_company_profile_tool, get_competitors_tool]

agent = create_tool_calling_agent(llm, tools, prompt)

To access intermediate steps, we could add this parameter: return_intermediate_steps=True

In [86]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose = True,  return_intermediate_steps=True)

In [87]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

agent_executor_w_memory = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

In [76]:
result = agent_executor_w_memory.invoke(
    {"input": "Tell me about AAPL"},
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 64cf8aed-fc78-4cbf-a588-f122fd530db9 not found for run 12900f53-a333-4be4-a4c3-b9f139eefdb9. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand rad

In [77]:
result

{'input': 'Tell me about AAPL',
 'history': [],
 'output': 'Apple Inc. (AAPL) is a technology company that designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories globally. Here are some key details about Apple Inc.:\n\n- **Sector:** Information Technology\n- **Primary Industry:** Technology Hardware, Storage and Peripherals\n- **Number of Employees:** 161,000\n- **Year Founded:** 1976\n- **Headquarters:** Cupertino, California\n- **Website:** [www.apple.com](www.apple.com)\n- **Market Cap:** $3,205,896,523,740\n- **Total Enterprise Value:** $3,148,149,523,740\n- **Analysts Recommendations:** 44\n- **EPS (Earnings Per Share):** $6.45\n- **Current Stock Price:** $213.25\n- **PE Ratio (Forward):** 32.35\n- **Dividend Yield:** 0.47%\n- **Dividend Rate:** $1.00\n- **Debt to Equity Ratio:** 140.97\n- **Total Debt to Capital:** 58.50\n- **Long-Term Debt to Equity:** 123.77\n- **Long-Term Debt to Capital:** 51.36\n\nApple offers a range of produ

As you can see, after we set return_intermediate_steps parameter to True, we could see the intermediate_steps which contains response from API. To access this and store into the memory, we need format_to_openai_functions to translate this into a message (FunctionMessage)

In [79]:
from langchain.agents.format_scratchpad import format_to_openai_functions
intermediate_steps = format_to_openai_functions(result['intermediate_steps'])
intermediate_steps

[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_7zWQPpNGoZxPr0xUNioE4sdd', 'function': {'arguments': '{"stock":"AAPL"}', 'name': 'get_company_profile'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-3.5-turbo-0125'}, id='run-4f256b65-a8ab-4bc3-bf1f-93ff7aee3075', tool_calls=[{'name': 'get_company_profile', 'args': {'stock': 'AAPL'}, 'id': 'call_7zWQPpNGoZxPr0xUNioE4sdd'}], tool_call_chunks=[{'name': 'get_company_profile', 'args': '{"stock":"AAPL"}', 'id': 'call_7zWQPpNGoZxPr0xUNioE4sdd', 'index': 0}]),
 FunctionMessage(content='{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Appl

In [83]:
result = agent_executor_w_memory.invoke(
    {"input": "Tell me about AAPL"},
    config={"configurable": {"session_id": "abc123"}},
)

history = get_session_history("abc123")
intermediate_steps = format_to_openai_functions(result['intermediate_steps'])

if len(intermediate_steps) > 0:
    history.add_message(intermediate_steps[1])

Parent run c458ba97-bf30-4629-b3ac-b6b383187d4c not found for run 477cfbb9-0f13-47f1-87b7-2e5cd1e7796f. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `get_company_profile` with `{'stock': 'AAPL'}`


{"data": [{"id": "AAPL", "tickerId": 146, "attributes": {"longDesc": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, and HomePod. It also provides AppleCare support and cloud services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. In addition, the company offers various services, such as Apple Arcade, a game subscription service; Apple Fitness+, a personalized fitness service; Apple Music, which offers users a curated listening experience with on-demand rad

In [84]:
result = agent_executor_w_memory.invoke(
    {"input": "Where is the headquarters located?"},
    config={"configurable": {"session_id": "abc123"}},
)

history = get_session_history("abc123")
intermediate_steps = format_to_openai_functions(result['intermediate_steps'])

if len(intermediate_steps) > 0:
    history.add_message(intermediate_steps[1])

Parent run fa5dda12-ed84-4985-a48e-542ed3c70806 not found for run c1207dc3-ed79-4813-8969-78bc6af15f75. Treating as a root run.




> Entering new AgentExecutor chain...
The headquarters of Apple Inc. is located at One Apple Park Way, Cupertino, California, 95014, United States.

> Finished chain.


Conclusion:

1. The agent understands that the context here is AAPL
2. Since we've saved the API response in memory, instead of calling the API again, it retrieves what was stored previously!

# Demo Final Chatbot using Chainlit